In [1]:
import os
from datetime import datetime

import numpy as np
import tensorflow as tf
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
from tensorflow import keras

In [2]:
LOG_BASE_DIR = './logs'

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

train_images  = train_images / 255.0
train_images = np.expand_dims(train_images, axis=-1)
print(f"train_images.shape = {train_images.shape}, train_labels.shape = {train_labels.shape}")
test_images = test_images / 255.0
test_images = np.expand_dims(test_images, axis=-1)
print(f"test_images.shape = {test_images.shape}, test_labels.shape = {test_labels.shape}")

train_images.shape = (60000, 28, 28, 1), train_labels.shape = (60000,)
test_images.shape = (10000, 28, 28, 1), test_labels.shape = (10000,)


In [4]:
def build_model(num_conv_blocks=2, conv_filter_size=(3, 3), learning_rate=0.001):
    print(f"build_model(): {locals()}")
    model = keras.models.Sequential()

    model.add(keras.layers.InputLayer(input_shape=(28, 28, 1)))

    for _ in range(num_conv_blocks):
        model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(keras.layers.MaxPooling2D(2, 2))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

## KerasRegressor

In [5]:
keras.backend.clear_session()

reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [6]:
reg.fit(
    train_images,
    train_labels, 
    epochs=10,
    batch_size=64,
    validation_data=(test_images, test_labels),
    verbose=1
)

build_model(): {'learning_rate': 0.001, 'conv_filter_size': (3, 3), 'num_conv_blocks': 2}
Epoch 1/10
938/938 [==============================] - 3s 3ms/step - loss: 0.4723 - accuracy: 0.8298 - val_loss: 0.3637 - val_accuracy: 0.8684
Epoch 2/10
938/938 [==============================] - 3s 4ms/step - loss: 0.3172 - accuracy: 0.8841 - val_loss: 0.3094 - val_accuracy: 0.8883
Epoch 3/10
938/938 [==============================] - 3s 4ms/step - loss: 0.2707 - accuracy: 0.9008 - val_loss: 0.3118 - val_accuracy: 0.8849
Epoch 4/10
938/938 [==============================] - 3s 4ms/step - loss: 0.2411 - accuracy: 0.9101 - val_loss: 0.2675 - val_accuracy: 0.9002
Epoch 5/10
938/938 [==============================] - 3s 3ms/step - loss: 0.2185 - accuracy: 0.9190 - val_loss: 0.2573 - val_accuracy: 0.9048
Epoch 6/10
938/938 [==============================] - 3s 3ms/step - loss: 0.1937 - accuracy: 0.9282 - val_loss: 0.2635 - val_accuracy: 0.9038
Epoch 7/10
938/938 [==============================] - 3s 3

## RandomizedSearchCV

In [7]:
keras.backend.clear_session()

params = {
    'num_conv_blocks': [1, 2, 3],
    'conv_filter_size': [(3, 3), (5, 5)],
    'learning_rate': [1e-3, 1e-4],
}

reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [8]:
search = RandomizedSearchCV(reg, params, n_iter=10, cv=3, verbose=2)

search.fit(
    train_images,
    train_labels, 
    epochs=10,
    batch_size=64,
    validation_data=(test_images, test_labels),
    verbose=0
)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] num_conv_blocks=3, learning_rate=0.001, conv_filter_size=(5, 5) .
build_model(): {'learning_rate': 0.001, 'conv_filter_size': (5, 5), 'num_conv_blocks': 3}


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


625/625 [==============================] - 1s 2ms/step - loss: 0.3040 - accuracy: 0.8888
[CV]  num_conv_blocks=3, learning_rate=0.001, conv_filter_size=(5, 5), total=  15.3s
[CV] num_conv_blocks=3, learning_rate=0.001, conv_filter_size=(5, 5) .
build_model(): {'learning_rate': 0.001, 'conv_filter_size': (5, 5), 'num_conv_blocks': 3}


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.3s remaining:    0.0s


625/625 [==============================] - 1s 2ms/step - loss: 0.3263 - accuracy: 0.8799
[CV]  num_conv_blocks=3, learning_rate=0.001, conv_filter_size=(5, 5), total=  15.1s
[CV] num_conv_blocks=3, learning_rate=0.001, conv_filter_size=(5, 5) .
build_model(): {'learning_rate': 0.001, 'conv_filter_size': (5, 5), 'num_conv_blocks': 3}
625/625 [==============================] - 1s 2ms/step - loss: 0.3261 - accuracy: 0.8837
[CV]  num_conv_blocks=3, learning_rate=0.001, conv_filter_size=(5, 5), total=  15.1s
[CV] num_conv_blocks=1, learning_rate=0.0001, conv_filter_size=(5, 5) 
build_model(): {'learning_rate': 0.0001, 'conv_filter_size': (5, 5), 'num_conv_blocks': 1}
625/625 [==============================] - 1s 2ms/step - loss: 0.3031 - accuracy: 0.8927
[CV]  num_conv_blocks=1, learning_rate=0.0001, conv_filter_size=(5, 5), total=  12.5s
[CV] num_conv_blocks=1, learning_rate=0.0001, conv_filter_size=(5, 5) 
build_model(): {'learning_rate': 0.0001, 'conv_filter_size': (5, 5), 'num_conv_bloc

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  7.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f9605ed8c18>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'conv_filter_size': [(3, 3), (5, 5)],
                                        'learning_rate': [0.001, 0.0001],
                                        'num_conv_blocks': [1, 2, 3]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [9]:
search.best_params_

{'conv_filter_size': (3, 3), 'learning_rate': 0.001, 'num_conv_blocks': 2}

In [10]:
search.best_score_

-0.2665172219276428

In [11]:
search.score(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.2761 - accuracy: 0.9117


-0.2760579586029053

In [12]:
model = search.best_estimator_.model
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.2761 - accuracy: 0.9117


[0.2760579586029053, 0.9117000102996826]